# Experimenting with Multivariate LSTM and indices...

I'm reading a datafile with all the REPSOL indices that we were computing using the 9th floor Spark processes. We will try to predict if the vlaue will go 'Up' or 'Down'.

In [6]:
import numpy
import math

import matplotlib.pyplot as plt
from pandas import read_csv, concat, DataFrame
from matplotlib import pyplot
from numpy import concatenate

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


### Read the input dataset

    Input <-  Path
    Output -> raw_dataset

In [7]:
def read_dataset(file_path):
    columNames = ['price','var-1','var-2','var-3','var-4','var-5','var-6','var-7','var-8',
                 'var-9','var-10','var-11','var-12','var-13','var-14','var-15']
    
    raw_dataset = read_csv(file_path, header='infer', delimiter=';', usecols=columNames)
    # Remove the first column as it contains the value we want to predict
    # dataset = raw_dataset.iloc[:, 1:]   
    return(raw_dataset)

### Series-To_Supervised

This function is KEY as it produces the array shaped with t-n look-back samples to feed the LSTM

In [8]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

### Scale > Reframe > Drop

Let's make things reproducible. We also convert anything that might not be a float to `float32`. Data in NN is normalized to produce equivalent responses in the different layers. We also do that in this chunk. Then, data is scaled to the range 0..1, reframed according to the syntax in `series-to-supervised` and finally, unuseful columns are removed.

In [9]:
def scale_reframe_drop(raw_dataset):
    # Data types conversion
    dataset = raw_dataset.astype('float32')

    # Reframe
    reframed = series_to_supervised(dataset, look_back, look_forward)

    # Drop 'num_features - 1' from the tail of each row, as I only want to keep the first one, 
    # which will be what I want to predict.
    num_features = int(reframed.shape[1] / (look_back + 1))
    num_cols = reframed.shape[1]
    cols_to_remove = [reframed.columns[col_idx] for col_idx in range(num_cols - num_features + 1, num_cols)]
    prepared_dataset = reframed.drop(cols_to_remove, axis=1)
    
    # Scale
    scaled = scaler.fit_transform(prepared_dataset)
    df = DataFrame(data=scaled[:,:], index=range(0,scaled.shape[0]), 
               columns=['var-{:d}'.format(i) for i in range(scaled.shape[1])])

    #return prepared_dataset
    print('scaled & reframed shape:', df.shape)
    return df

## Split in Training & Test

Split and reshape the dataset.

In [10]:
def split_train_test(prepared_dataset):
    # split into train and test sets
    values = prepared_dataset.values
    train_size = int(round(len(values) * training_set_proportion))
    train = values[0:train_size, :]
    test = values[train_size:, :]

    # Split into input and output.
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]
    
    # reshape input to be [samples, time steps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    
    return (train_X, train_y, test_X, test_y)

## Train the LSTM !
Define the LSTM parameters, and train it.

In [179]:
def train_model(train_X, train_y, test_X, test_y):
    # create and fit the LSTM network
    model = Sequential()
    if lstm_stateful is True:
        model.add(LSTM(neurons, batch_input_shape=(lstm_batch_size, train_X.shape[1], train_X.shape[2]), stateful=True))
    else:
        model.add(LSTM(neurons, input_shape=(train_X.shape[1], train_X.shape[2]), stateful=False))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    history = model.fit(train_X, train_y, epochs=num_epochs, batch_size=lstm_batch_size, 
                        validation_data=(test_X, test_y), verbose=keras_verbose_level, shuffle=lstm_shuffle)
    return model, history

def plot_model_training(history):
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.show()    

### Compute the error

Compute the error (RMSE) for training and test. Previously, the examples suggest to invert the results from prediction to use the same units than in the source data.

In [180]:
def invert_Y(test_X, Y):
    """
    Invert the Y vector. The way invert works requires to have a matrix with all
    the features in place. That's why I must concatenate the test_X, so that it
    can perform the matrix multiplication. To get only the Y, a column selection
    is done as a final step.
    """
    # Check if this Y vector is special (m,) and is not shaped correctly (m,1)
    if len(Y.shape) is 1:
        Y = Y.reshape((len(Y), 1))
    test_X_reshaped = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    # invert scaling the prediction to the original values, for forecast
    inv_Y = concatenate((Y, test_X_reshaped[:, 0:]), axis=1)
    inv_Y = scaler.inverse_transform(inv_Y)
    return inv_Y[:,0]


def predict(model, test_X, invert=True):
    """
    Make a prediction with the model over the test_X dataset as input.
    """ 
    yhat = model.predict(test_X, batch_size=lstm_batch_size)
    if invert is False:
        return yhat

    inv_yhat = invert_Y(test_X, yhat)
    return inv_yhat
    
    
def compute_tendency_errors(Y, Yhat):
    """
    Compute the error in tendency (sign of future value minus present value) when making a prediction.
    """
    num_errors = 0
    for idx in range(1, len(Y)):
        yhat_trend = numpy.sign(Yhat[idx]-Yhat[idx-1])
        y_trend = numpy.sign(Y[idx]-Y[idx-1])
        error = int(yhat_trend == y_trend)
        if error == 0:
            num_errors += 1
    return num_errors


def compute_error(inv_y, inv_yhat):
    """
    Compute the RMSE between the prediction and the actual values.
    """    
    rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
    return rmse, compute_tendency_errors(inv_y, inv_yhat)


def printout_errors(rmse, trend_errs, header=True):
    if header is True:
        print(' neurons | epochs | look_back | RMSE  | Trnd.E')
        print('---------|--------|-----------|-------|--------')
    print(' {:7d} | {:6d} | {:9d} | {:.03f} | {:02d}'.
          format(neurons, num_epochs, look_back, rmse, trend_errs))

### Plot actual values and predicted values.

Plot the whoel series, and the predicted values for the test set.

In [196]:
def plot_prediction(inv_y, inv_yhat, title='', timespan_length=0, num_rows=1, num_cols=1, plot_index=1):
    # Setup the plot
    if num_rows is 1 and num_cols is 1:
        plt.figure(num=None, figsize=(12, 8), dpi=80, facecolor='w', edgecolor='k')
    else:
        plt.subplot(num_rows, num_cols, plot_index)
    plt.gca().xaxis.grid(True, which='major', color='grey', linestyle='--')
    plt.ylabel('price')
    plt.title(title)
    
    # timespanlength is the total size of the plot, in case we want to share the
    # plot with the training set. If set to zero, it can be easily computed as 
    # the size of the Y or Y_hat sets.
    if timespan_length is 0:
        timespan_length = inv_y.shape[0]
    
    # place the values at the end of a large array (shifted by training samples elements)
    test_values = numpy.empty(timespan_length)
    #test_values[:] = numpy.nan
    test_values[-(len(inv_y)):] = inv_y
    
    # place the prediction as we did with test_values
    predicted_values = numpy.empty(timespan_length)
    #predicted_values[:] = numpy.nan
    predicted_values[-(len(inv_yhat)):] = inv_yhat
    
    # Plot everything
    plt.subplot(num_rows, num_cols, plot_index)
    test_plot = plt.plot(test_values, marker='o')
    plt.setp(test_plot, color='g', linewidth=3.0)
    prediction_plot = plt.plot(predicted_values, marker='o')
    plt.setp(prediction_plot, color='r', linewidth=1.0)
    
    # Print, if last plot or the only one.
    if num_rows is 1 and num_cols is 1:
        plt.show()

### Setup Hyper-Parameters and main pipeline function.

Set here all the pipeline parameters and the main pipeline function.
Scaler in range of (-1, +1) works better than in (0, 1).

In [11]:
numpy.random.seed(2)
file_path = '~/Documents/SideProjects/sailboatsfactory/data/ibex_1.csv'
scaler = MinMaxScaler(feature_range=(-1, 1))
lstm_stateful = True
lstm_shuffle = True
look_back = 10
num_epochs = 50
lstm_batch_size = 10
look_forward = 1
training_set_proportion = 0.9795
keras_verbose_level = 2

# read data
raw_dataset = read_dataset(file_path)
neurons = raw_dataset.shape[1]-1

def run_pipeline():
    # prepare it to fit the lstm input
    prepared_dataset = scale_reframe_drop(raw_dataset)
    neurons = (prepared_dataset.shape[1] - 1) * look_back
    # split dataset intraining and test
    train_X, train_y, test_X, test_y = split_train_test(prepared_dataset)
    # Train the model
    model, history = train_model(train_X, train_y, test_X, test_y)
    plot_model_training(history)
    # Make a prediction with the model and invert the actual test Y set.
    inv_yhat = predict(model, test_X, invert=True)
    inv_y = invert_Y(test_X, test_y)
    # Compute the errors
    rmse, trend_error = compute_error(inv_y, inv_yhat)
    
    return inv_y, inv_yhat, rmse, trend_error

## Optimal Prediction

One prediction based on 75 epochs, with look_acbk = 1 (my prediction for tomorrow will be based only in what happened today) seems the optimal one. In the title of the plot, the last term "T.E." stands for Trend Errors, which means the number of times the model failed to predict if the tendency was to increase the price or lower the price.

In [ ]:
y, yhat, rmse, trend_error = run_pipeline()
plot_prediction(y, yhat, title='LookBack={:d}, RMSE={:.02f}, T.E={:.02f}({:d}/{:d})'.
                format(look_back, rmse, (trend_error/(len(y)-1)), trend_error, len(y)-1))

## Set SIZE !!!!

In [12]:
raw_dataset = read_dataset(file_path)
print('raw_dataset:',raw_dataset.shape)
prepared_dataset = scale_reframe_drop(raw_dataset)
print('prepared_dataset:',prepared_dataset.shape)
train_X, train_y, test_X, test_y = split_train_test(prepared_dataset)
print('train_X:',train_X.shape,', train_y:', train_y.shape)
print('test_X:',test_X.shape,', test_y:', test_y.shape)

raw_dataset: (6860, 16)
scaled & reframed shape: (6850, 161)
prepared_dataset: (6850, 161)
train_X: (6710, 1, 160) , train_y: (6710,)
test_X: (140, 1, 160) , test_y: (140,)


In [28]:
params = dict()
params['file_path'] = '~/Documents/SideProjects/sailboatsfactory/data/ibex_1.csv'
params['scaler'] = MinMaxScaler(feature_range=(-1, 1))
params['lstm_stateful'] = True
params['lstm_shuffle'] = True
params['num_epochs'] = 50
params['lstm_batch_size'] = 10
params['look_forward'] = 1
params['keras_verbose_level'] = 2params['look_back'] = 2
params['raw_numrows'] = raw_dataset.shape[0]
params['lstm_batch_size'] = 50
params['test_numbatches'] = 2

params['raw_adjusted_numrows'] = params['raw_numrows'] - ((params['raw_numrows'] - params['look_back']) % params['lstm_batch_size'])
params['reframed_numrows'] = (params['raw_adjusted_numrows'] - params['look_back'])
params['train_numrows'] = params['reframed_numrows'] - (params['test_numbatches'] * params['lstm_batch_size'])
params['test_numrows']  = (params['test_numbatches'] * params['lstm_batch_size'])

# --

print('raw_numrows:', params['raw_numrows'])
print('raw_adjusted_numrows:', params['raw_adjusted_numrows'])
print('ref_size:', params['reframed_numrows'])
print('train_numrows:', params['train_numrows'])
print('test_numrows', params['test_numrows'])

raw_numrows: 6860
raw_adjusted_numrows: 6852
ref_size: 6850
train_numrows: 6830
test_numrows 20
